In [50]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.utils import resample

In [51]:
df = pd.read_csv("OnlineNewsPopularity.csv")
df = df.drop(columns=["url", " timedelta"])

In [52]:
df[' shares'].values[df[' shares'].values < 1400] = 0 #unpopular
df[' shares'].values[df[' shares'].values >= 1400] = 1 #popular
print (sum(df[' shares'].values==0))
print (sum(df[' shares'].values==1))
print (sum(df[' shares'].values==1400))

18490
21154
0


In [53]:
#results = pd.DataFrame(columns=["Algorithm", "AUC"])
#results = pd.DataFrame(columns=["Algorithm", "Start", "End", "AUC","Avg AUC"])
results = pd.DataFrame(columns=["Algorithm","Train Start", "Train End", "Test Start", "Test End", "AUC","Avg AUC"])

In [7]:
for i in range(0, (len(df)-10000), 1000): #29000-39000 train
    if((i+11000)<len(df)): 
        print ("Train",i,i+10000)
        print ("Test",i+10000,i+11000)
    else:
        print("Train",i,i+10000)
        print("Test",i+10000,len(df))


Train 0 10000
Test 10000 11000
Train 1000 11000
Test 11000 12000
Train 2000 12000
Test 12000 13000
Train 3000 13000
Test 13000 14000
Train 4000 14000
Test 14000 15000
Train 5000 15000
Test 15000 16000
Train 6000 16000
Test 16000 17000
Train 7000 17000
Test 17000 18000
Train 8000 18000
Test 18000 19000
Train 9000 19000
Test 19000 20000
Train 10000 20000
Test 20000 21000
Train 11000 21000
Test 21000 22000
Train 12000 22000
Test 22000 23000
Train 13000 23000
Test 23000 24000
Train 14000 24000
Test 24000 25000
Train 15000 25000
Test 25000 26000
Train 16000 26000
Test 26000 27000
Train 17000 27000
Test 27000 28000
Train 18000 28000
Test 28000 29000
Train 19000 29000
Test 29000 30000
Train 20000 30000
Test 30000 31000
Train 21000 31000
Test 31000 32000
Train 22000 32000
Test 32000 33000
Train 23000 33000
Test 33000 34000
Train 24000 34000
Test 34000 35000
Train 25000 35000
Test 35000 36000
Train 26000 36000
Test 36000 37000
Train 27000 37000
Test 37000 38000
Train 28000 38000
Test 38000 3900

In [31]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
aucs = []
for i in range(0, (len(df)-10000), 1000):
    if((i+11000)<len(df)):
        print ("Train",i,i+10000)
        print ("Test",i+10000,i+11000)
        train=df.iloc[i:i+10000,:] 
        test=df.iloc[i+10000:i+11000,:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Start":[i], "End":[i+10000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
    else:
        print ("Train",i,i+10000)
        print ("Test",i+10000,len(df))
        train=df.iloc[i:i+10000,:] 
        test=df.iloc[i+10000:len(df),:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Start":[i], "End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 0 10000
Test 10000 11000
[22:05:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7031482921770692
Train 1000 11000
Test 11000 12000
[22:05:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7217539230065058
Train 2000 12000
Test 12000 13000
[22:06:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

Average: 0.7263469136124788
Train 28000 38000
Test 38000 39000
[22:10:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7258893784385675
Train 29000 39000
Test 39000 39644
[22:10:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7257251176690769


In [32]:
results.to_csv("xgb_roll.csv")

In [60]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
aucs = []
for i in range(0, (len(df)-10000), 2000):
    if((i+12000)<len(df)):
        print ("Train",i,i+10000)
        print ("Test",i+10000,i+12000)
        train=df.iloc[i:i+10000,:] 
        test=df.iloc[i+10000:i+12000,:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Start":[i], "End":[i+10000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
    else:
        print ("Train",i,i+10000)
        print ("Test",i+10000,len(df))
        train=df.iloc[i:i+10000,:] 
        test=df.iloc[i+10000:len(df),:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Start":[i], "End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 0 10000
Test 10000 12000
[23:31:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7186081587205183
Train 2000 12000
Test 12000 14000
[23:31:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7112217608385918
Train 4000 14000
Test 14000 16000
[23:31:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

Average: 0.7232281271601505
Train 26000 36000
Test 36000 38000
[23:33:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7237652405833038
Train 28000 38000
Test 38000 39644
[23:33:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7226617248471933


In [61]:
results.to_csv("xgb_roll2.csv")

In [20]:
for j in range(5000,22500,2500):
    for i in range(0, (len(df)-j), 1000): 
        if((i+j+1000)<len(df)): 
            print ("Train",i,i+j)
            print ("Test",i+j,i+(j+1000))
        else:
            print("Train",i,i+j)
            print("Test",i+j,len(df))


Train 0 5000
Test 5000 6000
Train 1000 6000
Test 6000 7000
Train 2000 7000
Test 7000 8000
Train 3000 8000
Test 8000 9000
Train 4000 9000
Test 9000 10000
Train 5000 10000
Test 10000 11000
Train 6000 11000
Test 11000 12000
Train 7000 12000
Test 12000 13000
Train 8000 13000
Test 13000 14000
Train 9000 14000
Test 14000 15000
Train 10000 15000
Test 15000 16000
Train 11000 16000
Test 16000 17000
Train 12000 17000
Test 17000 18000
Train 13000 18000
Test 18000 19000
Train 14000 19000
Test 19000 20000
Train 15000 20000
Test 20000 21000
Train 16000 21000
Test 21000 22000
Train 17000 22000
Test 22000 23000
Train 18000 23000
Test 23000 24000
Train 19000 24000
Test 24000 25000
Train 20000 25000
Test 25000 26000
Train 21000 26000
Test 26000 27000
Train 22000 27000
Test 27000 28000
Train 23000 28000
Test 28000 29000
Train 24000 29000
Test 29000 30000
Train 25000 30000
Test 30000 31000
Train 26000 31000
Test 31000 32000
Train 27000 32000
Test 32000 33000
Train 28000 33000
Test 33000 34000
Train 29000 

In [66]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
for j in range(5000,22500,2500):
    aucs = []
    for i in range(0, (len(df)-j), 1000):
        if((i+j+1000)<len(df)):
            print ("Train",i,i+j)
            print ("Test",i+j,i+(j+1000))
            train=df.iloc[i:i+j,:] 
            test=df.iloc[i+j:i+(j+1000),:] 
            model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
            auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
            #print (aucs)
            print("Average: " + str(sum(aucs)/len(aucs)))
            auc=str(sum(aucs)/len(aucs))
            results=results.append({"Algorithm": ["XGBoost"], "Train Start":[i], "Train End":[i+j], "Test Start":[i+j], "Test End":[i+j+1000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
        else:
            print ("Train",i,i+j)
            print ("Test",i+j,len(df))
            train=df.iloc[i:i+j,:] 
            test=df.iloc[i+j:len(df),:] 
            model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
            auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
            #print (aucs)
            print("Average: " + str(sum(aucs)/len(aucs)))
            auc=str(sum(aucs)/len(aucs))
            results=results.append({"Algorithm": ["XGBoost"], "Train Start":[i], "Train End":[i+j], "Test Start":[i+j], "Test End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 0 5000
Test 5000 6000
[11:48:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.6453055141579732
Train 1000 6000
Test 6000 7000
[11:48:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.6629430775017574
Train 2000 7000
Test 7000 8000
[11:48:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

Average: 0.7393097578381441
Train 18000 38000
Test 38000 39000
[12:22:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7383958904793493
Train 19000 39000
Test 39000 39644
[12:22:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.73751002172081


In [67]:
results.to_csv("cont4_xgb.csv")

In [34]:
for i in range(5000, (len(df)-500), 1000): 
    if((i+1000)<len(df)): 
        print ("Train",1,i)
        print ("Test",i,i+1000)
    else:
        print("Train",1,i)
        print("Test",i,len(df))


Train 1 5000
Test 5000 6000
Train 1 6000
Test 6000 7000
Train 1 7000
Test 7000 8000
Train 1 8000
Test 8000 9000
Train 1 9000
Test 9000 10000
Train 1 10000
Test 10000 11000
Train 1 11000
Test 11000 12000
Train 1 12000
Test 12000 13000
Train 1 13000
Test 13000 14000
Train 1 14000
Test 14000 15000
Train 1 15000
Test 15000 16000
Train 1 16000
Test 16000 17000
Train 1 17000
Test 17000 18000
Train 1 18000
Test 18000 19000
Train 1 19000
Test 19000 20000
Train 1 20000
Test 20000 21000
Train 1 21000
Test 21000 22000
Train 1 22000
Test 22000 23000
Train 1 23000
Test 23000 24000
Train 1 24000
Test 24000 25000
Train 1 25000
Test 25000 26000
Train 1 26000
Test 26000 27000
Train 1 27000
Test 27000 28000
Train 1 28000
Test 28000 29000
Train 1 29000
Test 29000 30000
Train 1 30000
Test 30000 31000
Train 1 31000
Test 31000 32000
Train 1 32000
Test 32000 33000
Train 1 33000
Test 33000 34000
Train 1 34000
Test 34000 35000
Train 1 35000
Test 35000 36000
Train 1 36000
Test 36000 37000
Train 1 37000
Test 370

In [84]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
aucs = []
for i in range(5000, (len(df)-500), 1000):
    if((i+1000)<len(df)):
        print ("Train",1,i)
        print ("Test",i,i+1000)
        train=df.iloc[1:i,:] 
        test=df.iloc[i:i+1000,:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Train Start":[1], "Train End":[i], "Test Start":[i], "Test End":[i+1000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
    else:
        print ("Train",1,i)
        print ("Test",i,len(df))
        train=df.iloc[1:i,:] 
        test=df.iloc[i:len(df),:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Train Start":[1], "Train End":[i], "Test Start":[i], "Test End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 1 5000
Test 5000 6000
[12:54:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.649636028429192
Train 1 6000
Test 6000 7000
[12:55:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.6587471231850957
Train 1 7000
Test 7000 8000
[12:55:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...


Average: 0.7264598860678368
Train 1 38000
Test 38000 39000


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:04:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Average: 0.726585430117776
Train 1 39000
Test 39000 39644
[13:05:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Average: 0.7266871525950771


In [85]:
results.to_csv("cont5_xgb.csv")

In [5]:
for i in range(0, (len(df)-32000), 1000): 
    if((i+33000)<len(df)): 
        print ("Train",i,i+32000)
        print ("Test",i+32000,i+33000)
    else:
        print("Train",i,i+32000)
        print("Test",i+32000,len(df))


Train 0 32000
Test 32000 33000
Train 1000 33000
Test 33000 34000
Train 2000 34000
Test 34000 35000
Train 3000 35000
Test 35000 36000
Train 4000 36000
Test 36000 37000
Train 5000 37000
Test 37000 38000
Train 6000 38000
Test 38000 39000
Train 7000 39000
Test 39000 39644


In [24]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
aucs = []
for i in range(0, (len(df)-32000), 1000):
    if((i+33000)<len(df)):
        print ("Train",i,i+32000)
        print ("Test",i+32000,i+33000)
        train=df.iloc[i:i+32000,:] 
        test=df.iloc[i+32000:i+33000,:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Train Start":[i], "Train End":[i+32000], "Test Start":[i+32000], "Test End":[i+33000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
    else:
        print ("Train",i,i+32000)
        print ("Test",i+32000,len(df))
        train=df.iloc[i:i+32000,:] 
        test=df.iloc[i+32000:len(df),:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Train Start":[i], "Train End":[i+32000], "Test Start":[i+32000], "Test End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 0 32000
Test 32000 33000
[21:43:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7935684855793388
Train 1000 33000
Test 33000 34000
[21:43:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Average: 0.781466791372633
Train 2000 34000
Test 34000 35000


...

Average: 0.7470835979296067
Train 6000 38000
Test 38000 39000
[21:46:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Average: 0.7444798826549662
Train 7000 39000
Test 39000 39644
[21:46:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7436372720778754


In [25]:
results.to_csv("cont4_32k.csv")

In [31]:
j=0
for i in range(5000, (len(df)-500), 1000): 
    if(i<=20000): 
        print ("Train",1,i)
        print ("Test",i,i+1000)
    else:
        j=j+1000
        if((i+1000)<len(df)):
            print("Train",j,i)
            print("Test",i,i+1000)
        else:
            print("Train",j,i)
            print("Test",i,len(df))


Train 1 5000
Test 5000 6000
Train 1 6000
Test 6000 7000
Train 1 7000
Test 7000 8000
Train 1 8000
Test 8000 9000
Train 1 9000
Test 9000 10000
Train 1 10000
Test 10000 11000
Train 1 11000
Test 11000 12000
Train 1 12000
Test 12000 13000
Train 1 13000
Test 13000 14000
Train 1 14000
Test 14000 15000
Train 1 15000
Test 15000 16000
Train 1 16000
Test 16000 17000
Train 1 17000
Test 17000 18000
Train 1 18000
Test 18000 19000
Train 1 19000
Test 19000 20000
Train 1 20000
Test 20000 21000
Train 1000 21000
Test 21000 22000
Train 2000 22000
Test 22000 23000
Train 3000 23000
Test 23000 24000
Train 4000 24000
Test 24000 25000
Train 5000 25000
Test 25000 26000
Train 6000 26000
Test 26000 27000
Train 7000 27000
Test 27000 28000
Train 8000 28000
Test 28000 29000
Train 9000 29000
Test 29000 30000
Train 10000 30000
Test 30000 31000
Train 11000 31000
Test 31000 32000
Train 12000 32000
Test 32000 33000
Train 13000 33000
Test 33000 34000
Train 14000 34000
Test 34000 35000
Train 15000 35000
Test 35000 36000
Tr

In [54]:
#model = DecisionTreeClassifier(random_state = 42, max_depth = 7)
#model = KNeighborsClassifier(n_neighbors = 11)
#model = RandomForestClassifier(random_state = 42, n_estimators = 1000, max_depth = 6)
model = XGBClassifier(random_state = 42, n_estimators = 500, eta = 0.1, max_depth = 3)
aucs = []
j=0
for i in range(5000, (len(df)-500), 1000): 
    if(i<=20000): 
        print ("Train",1,i)
        print ("Test",i,i+1000)
        train=df.iloc[1:i,:] 
        test=df.iloc[i:i+1000,:] 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
        auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
        #print (aucs)
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc=str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["XGBoost"], "Train Start":[1], "Train End":[i], "Test Start":[i], "Test End":[i+1000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
    else:
        j=j+1000
        if((i+1000)<len(df)):
            print("Train",j,i)
            print("Test",i,i+1000)
            train=df.iloc[j:i,:] 
            test=df.iloc[i:i+1000,:] 
            model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
            auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
            #print (aucs)
            print("Average: " + str(sum(aucs)/len(aucs)))
            auc=str(sum(aucs)/len(aucs))
            results=results.append({"Algorithm": ["XGBoost"], "Train Start":[j], "Train End":[i], "Test Start":[i], "Test End":[i+1000], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)
        else:
            print("Train",j,i)
            print("Test",i,len(df))
            train=df.iloc[j:i,:] 
            test=df.iloc[i:len(df),:] 
            model.fit(train.iloc[:, :-1], train.iloc[:, -1]) 
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr"))
            auc_single=roc_auc_score(test.iloc[:, -1], predictions[:, 1], multi_class="ovr")
            #print (aucs)
            print("Average: " + str(sum(aucs)/len(aucs)))
            auc=str(sum(aucs)/len(aucs))
            results=results.append({"Algorithm": ["XGBoost"], "Train Start":[j], "Train End":[i], "Test Start":[i], "Test End":[len(df)], "AUC":[auc_single], "Avg AUC": [auc]}, ignore_index=True)

Train 1 5000
Test 5000 6000
[17:45:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.649636028429192
Train 1 6000
Test 6000 7000
[17:45:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.6587471231850957
Train 1 7000
Test 7000 8000
[17:46:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

Average: 0.7255868727080304
Train 18000 38000
Test 38000 39000
[17:54:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7254797963937073
Train 19000 39000
Test 39000 39644
[17:54:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Average: 0.7253426169341317


In [55]:
results.to_csv("cont6_xgb.csv")